# SPR 2026 - Word2Vec + XGBoost

**Experimento Word2Vec com XGBoost**

XGBoost com tuning agressivo para embeddings densos.

---
**CONFIGURAÇÃO OFFLINE:**
1. No Kaggle, vá em Settings → Internet → **OFF**
2. Gensim e XGBoost já estão pré-instalados no Kaggle
---

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.utils.class_weight import compute_sample_weight
from gensim.models import Word2Vec
import re
import warnings
warnings.filterwarnings('ignore')

SEED = 42
EMBEDDING_DIM = 100
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
np.random.seed(SEED)
print('Bibliotecas carregadas!')

In [ ]:
# Carregar dados
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')

print(f'Train: {train.shape}')
print(f'Test: {test.shape}')

In [ ]:
# Preprocessamento
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-záàâãéèêíïóôõöúçñ\s]', ' ', text)
    return text.split()

train['tokens'] = train['report'].apply(preprocess)
test['tokens'] = test['report'].apply(preprocess)

print(f'Exemplo: {train["tokens"].iloc[0][:10]}')

In [ ]:
# Treinar Word2Vec
all_texts = train['tokens'].tolist() + test['tokens'].tolist()

w2v = Word2Vec(
    sentences=all_texts,
    vector_size=EMBEDDING_DIM,
    window=5,
    min_count=2,
    workers=4,
    epochs=10,
    seed=SEED
)

print(f'Vocabulário: {len(w2v.wv)} palavras')

In [ ]:
# Converter textos em embeddings (Mean Pooling)
def text_to_embedding(tokens, model, dim):
    vectors = [model.wv[w] for w in tokens if w in model.wv]
    if len(vectors) == 0:
        return np.zeros(dim)
    return np.mean(vectors, axis=0)

X_train = np.array([text_to_embedding(t, w2v, EMBEDDING_DIM) for t in train['tokens']])
X_test = np.array([text_to_embedding(t, w2v, EMBEDDING_DIM) for t in test['tokens']])
y_train = train['target'].values

print(f'X_train shape: {X_train.shape}')

In [ ]:
# Sample weights para classes desbalanceadas
sample_weights = compute_sample_weight('balanced', y_train)

# Treinar XGBoost
model = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=3,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=SEED,
    n_jobs=-1,
    use_label_encoder=False,
    eval_metric='mlogloss'
)

model.fit(X_train, y_train, sample_weight=sample_weights)
print('Modelo XGBoost treinado!')

In [ ]:
# Predições e submissão
predictions = model.predict(X_test)

submission = pd.DataFrame({
    'ID': test['ID'],
    'target': predictions
})

submission.to_csv('submission.csv', index=False)

print('submission.csv criado!')
print(submission['target'].value_counts().sort_index())